# env setup

In [14]:
import os
LANGCHAIN_API_KEY = os.getenv(key="LANGCHAIN_API_KEY")
LANGCHAIN_ENDPOINT = os.getenv(key="LANGCHAIN_ENDPOINT")
LANGCHAIN_TRACING_V2 = os.getenv(key="LANGCHAIN_TRACING_V2")
from dotenv import load_dotenv
load_dotenv()
groq_api_key = os.environ['GROQ_API_KEY']

# define llms

In [15]:
from langchain_community.chat_models import ChatOllama
phi_llm = ChatOllama(model='phi',temperature=0.1,timeout=300)
gemma_llm = ChatOllama(model='gemma:2b',temperature=0.1,timeout=300)

In [16]:
llm = phi_llm

# DB: Connect to a SQLite DB.

In [17]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///nba_roster.db", sample_rows_in_table_info= 0)

In [18]:
from langchain_core.prompts import ChatPromptTemplate
# Update the template based on the type of SQL Database like MySQL, Microsoft SQL Server and so on
template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""


In [19]:
query = "SELECT Team FROM nba_roster WHERE NAME = 'Klay Thompson'"
db.run(query)

"[('Golden State Warriors',)]"

In [20]:
def get_schema(_):
    return db.get_table_info()

def run_query(query):
    return db.run(query)

Query a SQL Database

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
# Update the template based on the type of SQL Database like MySQL, Microsoft SQL Server and so on
template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:"""  #noqa:E501


In [ ]:
template

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Given an input question, convert it to a SQL query. No pre-amble."),
    ("human", template)
])

In [ ]:
prompt

# work with different prompts from hub

In [ ]:
# Update the template based on the type of SQL Database like MySQL, Microsoft SQL Server and so on
hub_template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:
SQLResult: "Result of the SQLQuery"
Answer: "Final answer should be in a statement."
"""


In [ ]:
# hub_prompt = ChatPromptTemplate.from_messages([
#     ("system", "Given an input question, convert it to a SQL query. No pre-amble."),
#     ("human", hub_template)
# ])
# hub_prompt
# Pull down prompt
from langchain import hub
hub_prompt = hub.pull("rlm/text-to-sql")

In [ ]:
hub_prompt

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
inputs = {
    "table_info": lambda x: db.get_table_info(),
    "input": lambda x: x["question"],
    "few_shot_examples": lambda x: "",
    "dialect": lambda x: db.dialect,
}

sql_response = (
    inputs
    | hub_prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [ ]:
sql_response.invoke({"question": "What team is Klay Thompson on?"})

In [ ]:
# Chain to query
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
sql_response = (RunnablePassthrough.assign(schema=get_schema)
                | hub_prompt
                | llm.bind(stop=["\nSQLResult:"])
                | StrOutputParser()
            )

In [ ]:
sql_response.invoke({"question": "What team is Klay Thompson on?"})

Chat with a SQL DB

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
template = """Given an input question, convert it to a SQL query. No pre-amble. Based on the table schema below, write a SQL query that would answer the user's question:
{schema}
"""

In [ ]:
prompt = ChatPromptTemplate.from_messages(

    [
        ("system", template),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)

In [ ]:
# Chain to query with memory
from langchain_core.runnables import RunnableLambda

sql_chain = (
    RunnablePassthrough.assign(
        schema=get_schema,
        history=RunnableLambda(lambda x: memory.load_memory_variables(x)["history"]),
    )
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

def save(input_output):
    output = {"output": input_output.pop("output")}
    memory.save_context(input_output, output)

sql_response_memory = RunnablePassthrough.assign(output=sql_chain) | save
sql_response_memory.invoke({"question": "What team is Klay Thompson on?"})

# New sql_ollama from langchain

In [21]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
template = """Based on the table schema below, write a SQL query that would answer the user's question:
{schema}

Question: {question}
SQL Query:
Answer: "Final answer should be SQL Query."
"""  # noqa: E501

In [22]:

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Given an input question, convert it to a SQL query. No pre-amble."),
        MessagesPlaceholder(variable_name="history"),
        ("human", template),
    ]
)

In [23]:
prompt

ChatPromptTemplate(input_variables=['history', 'question', 'schema'], input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given an input question, convert it to a SQL query. No pre-amble.')), MessagesPlaceholder(variable_name='history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question', 'schema'], template='Based on the table schema below, write a SQL query that would answer the user\'s question:\n{schema}\n\nQuestion: {question}\nSQL Query:\nAnswer: "Final answer should be SQL Query."\n'))])

In [24]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(return_messages=True)

In [25]:
# Chain to query with memory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

sql_chain = (
    RunnablePassthrough.assign(
    schema=get_schema,
    history=RunnableLambda(lambda x: memory.load_memory_variables(x)["history"]),
    )
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [26]:
# sql_chain.invoke({"question": "What team is Klay Thompson on?"})

In [27]:
def save(input_output):
    output = {"output": input_output.pop("output")}
    memory.save_context(input_output, output)
    return output["output"]

In [28]:
sql_response_memory = RunnablePassthrough.assign(output=sql_chain) | save

In [29]:
# sql_response_memory.invoke({"question": "What team is Klay Thompson on?"})

In [30]:
# Chain to answer
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""  # noqa: E501

In [31]:
prompt_response = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Given an input question and SQL response, convert it to a natural "
            "language answer and also provide the SQL response in asnwer. No pre-amble.",
        ),
        ("human", template),
    ]
)


In [32]:
# Supply the input types to the prompt
from langchain_core.pydantic_v1 import BaseModel
class InputType(BaseModel):
    question: str

In [33]:
chain = (
    RunnablePassthrough.assign(query=sql_response_memory).with_types(
    input_type=InputType
    )
    | RunnablePassthrough.assign(
        schema=get_schema,
        response=lambda x: db.run(x["query"])
    )
    | prompt_response
    | llm
)

In [ ]:
chain.invoke({"question": "What team is Klay Thompson on?"})